In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
class Point:

    def __init__(self, x, y, z=0):
        self._x = self._check_xyz_data(x)
        self._y = self._check_xyz_data(y)
        self._z = self._check_xyz_data(z)

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    @property
    def z(self):
        return self._z

    @x.setter
    def x(self, value):
        self._x = self._check_xyz_data(value)

    @y.setter
    def y(self, value):
        self._y = self._check_xyz_data(value)

    @z.setter
    def z(self, value):
        self._z = self._check_xyz_data(value)

    @staticmethod
    def _check_xyz_data(value):
        if isinstance(value, float):
            return value
        elif isinstance(value, int):
            return float(value)
        else:
            raise ValueError(f"Должно быть число! Передан тип: {type(value)} - {value}")

    def __str__(self):
        return f"Point (x={self._x}, y={self._y}, x={self._z})"

In [ ]:
class Circle:

    def __init__(self, c_point: Point, radius):
        self._c_point = c_point
        self._radius = radius

    @property
    def x0(self):
        return self._c_point.x

    @property
    def y0(self):
        return self._c_point.y

    @property
    def r(self):
        return self._radius

    def __str__(self):
        return f"Circle (x0={self.x0}, y0={self.y0}, R={self.r})"

In [ ]:
class Graphic:

    _drawable_obj = []

    @classmethod
    def plot(cls):
        fig, ax = plt.subplots()
        for obj in cls._drawable_obj:
            obj.add_obj_to_ax(ax)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.grid()
        plt.axis("equal")
        plt.show()

    @classmethod
    def add_obj(cls, obj):
        cls._drawable_obj.append(obj)

    @classmethod
    def del_obj(cls, obj):
        cls._drawable_obj.remove(obj)

In [ ]:
from abc import abstractmethod

class Drawable:

    def __init__(self):
        Graphic.add_obj(self)

    @abstractmethod
    def add_obj_to_ax(self, ax):
        pass

In [ ]:
class DrawablePoint(Point, Drawable):

    def __init__(self, x, y, z=0):
        Point.__init__(self, x, y, z)
        Drawable.__init__(self)

    def add_obj_to_ax(self, ax):
        ax.scatter(self.x, self.y)

In [ ]:
class DrawableCircle(Circle, Drawable):

    def __init__(self, c_point: Point, radius):
        Circle.__init__(self, c_point, radius)
        Drawable.__init__(self)

    def add_obj_to_ax(self, ax):
        circle = plt.Circle((self.x0, self.y0), self.r, color='blue', fill=False, linewidth=2)
        ax.add_patch(circle)

In [ ]:
class NewCircle(DrawableCircle):

    def __init__(self, c_point: Point, radius):
        super().__init__(c_point, radius)

    def fit_circle_to_points(self, points, fitter_obj, 
                             max_iteration=50,
                             max_tolerance=1e-4,
                             print_log=True,
                             fit_inplace=True,
                             ):
        fitter_obj = fitter_obj(self)
        fitter_obj.fit_to_points(circle=self, 
                                 points=points, 
                                 max_iteration=max_iteration, 
                                 max_tolerance=max_tolerance, 
                                 print_log=print_log)
        new_c_point = Point(fitter_obj.x0, fitter_obj.y0)
        if fit_inplace:
            self._c_point = new_c_point
            self._radius = fitter_obj.r
            return self
        else:
            return NewCircle(new_c_point, fitter_obj.r)

In [ ]:
correct_circle = NewCircle(c_point=Point(100, 100), radius=10)

num_points = 20
deviation = 1
angles = np.linspace(0, 2 * np.pi, num_points)

points = []
for angle in angles:
    # Случайное отклонение радиуса в пределах ±deviation
    r = correct_circle.r + np.random.uniform(-deviation, deviation)
    # Координаты точки
    x = correct_circle.x0 + r * np.cos(angle)
    y = correct_circle.y0 + r * np.sin(angle)
    points.append(DrawablePoint(x, y))

Graphic.plot()

In [ ]:
class LSMCircleFitter:

    def __init__(self, circle):
        self.x0 = circle.x0
        self.y0 = circle.y0
        self.r = circle.r

    def _get_a_matrix(self, points):
        a_lst = []
        for point in points:
            r = ((point.x - self.x0) ** 2 + (point.y - self.y0) ** 2) ** 0.5
            a = -(point.x - self.x0) / r
            b = -(point.y - self.y0) / r
            c = -1
            a_lst.append([a, b, c])
        return np.array(a_lst)

    def _get_l_vector(self, points):
        l_vct = []
        for point in points:
            r = ((point.x - self.x0) ** 2 + (point.y - self.y0) ** 2) ** 0.5
            l = r - self.r
            l_vct.append([l])
        return np.array(l_vct)

    def _get_dt(self, points):
        a = self._get_a_matrix(points)
        l = self._get_l_vector(points)
        n = a.T @ a
        q = np.linalg.inv(n)
        t = -q @ a.T @ l
        return t

    def fit_to_points(self, circle, points, max_iteration=50, max_tolerance=1e-4, print_log=True):
        for i in range(max_iteration):
            t = self._get_dt(points)
            self.x0 += t[0][0]
            self.y0 += t[1][0]
            self.r += t[2][0]
            if print_log:
                print("*" * 25, f"iteration - {i}","*" * 25)
                print(t.T)
                print(self)
            if abs(max(t)[0]) < max_tolerance:
                break

In [ ]:
print(correct_circle)

fit_circle = correct_circle.fit_circle_to_points(points, LSMCircleFitter, fit_inplace=True)
print(fit_circle)

In [ ]:
Graphic.plot()